# Notebook 29 — Inference Pipeline: Predicting Opening Performance for New Players

## Purpose

This notebook demonstrates the complete inference pipeline for making opening recommendations to players **not** in our training set.
We use the 1,000 holdout players (reserved in notebook 28) to validate our approach before deploying to production.

**Production Use**: This pipeline will be used on the website to:
- Fetch a player's game history from the Lichess API
    - Note that for the testing in this notebook, we're just getting an untouched holdout player from our local DB.
- Transform their opening statistics into model-ready features
- Generate personalized opening recommendations
- Display predictions with confidence scores

**Development Focus**: We're building **granular, reusable functions** that can be easily adapted from local DB testing to production API integration.

---

## Pipeline Steps

### 1. **Load Holdout Player Data**
- Select a holdout player from the database (never seen during training)
- Extract their complete opening statistics (wins, draws, losses per opening)
- Retrieve player metadata (rating, name, etc.)
- Verify data quality and completeness
- Will use the same player every time this is run, for testing

### 2. **Transform Data for Model Input**
- Calculate raw performance scores: `(wins + 0.5 × draws) / total_games`
- Completely ignore openings not in training set
    - We got rid of those for good reasons, not helpful/unrepresentative/too generic
- Apply hierarchical Bayesian shrinkage toward opening-specific means
- Normalize player rating using training set parameters (z-score)
- Remap database IDs to training IDs (sequential 0-based indices)
- Encode ECO codes into categorical features (letter and number)
- Convert to PyTorch tensors
    - player id (not sure this is needed), opening ids, eco letters and numbers, rating_z

### 3. **Generate Predictions**
- Load trained model and all required artifacts (mappings, normalization params, etc.)
- Feed transformed data through the model
- Generate predicted performance scores for all valid openings
- Separate predictions into: openings player has played vs. new recommendations

### 4. **Analyze and Display Results**
- Compare predictions to actual performance (for openings player has played)
- Rank and display top opening recommendations
- Visualize prediction quality and confidence
- Save predictions for later analysis

---

**Note**: This notebook only performs inference—the model weights remain fixed. We are validating the deployment pipeline, not retraining.

**Note about one possible issue**: I believe we will need opening statistics from the training data to perform Bayesian shrinkage on win rates. We may not have that readily available; I'll need to double check our artifacts. But it should be fairly easy to compile.

## Required Model Artifacts

In this notebook (and in production), we will need to load various model artifact files to help in data processing.

I currently believe that we only need very small files (< 1MB total), which is very doable on the client or server.

From the trained model's artifact directory, we'll need:

### **Core Model & Mappings**
- **`best_model.pt`** (8.3 MB) - Trained PyTorch model weights for inference
  - This will live on the HuggingFace space where we host our model, so we don't need to slow down production by loading it in.
- **Note**: Need to make sure we're using a valid player id, if any. It might have to be the next sequential player id that the model's dataset would expect? Not sure, need to investigate.
- **`opening_id_mappings.json`** (96 KB) - Maps database opening IDs → training IDs (0-based sequential)
  - *Why*: Same as above - embeddings require contiguous indices

### **Normalization & Encoding**
- **`rating_normalization.json`** (4 KB) - Contains `rating_mean` and `rating_std` from training
  - *Why*: Must normalize new player ratings with exact same parameters: `z = (rating - mean) / std`
- **`eco_encodings.json`** (4 KB) - Maps ECO codes to integers (letter: A-E → 0-4, numbers → sequential ints)
  - *Why*: Model expects categorical integers, not raw ECO strings like "B02"

### **Model Configuration**
- **`hyperparameters.json`** (4 KB) - NUM_FACTORS (40), NUM_PLAYERS, NUM_OPENINGS, embedding dimensions
  - *Why*: Need exact architecture specs to instantiate the model correctly
- **`model_architecture.json`** (4 KB) - Complete model structure details
  - *Why*: Documents the model class and layer configurations

### **Reference Data**
- **`opening_mappings.csv`** (168 KB) - Full opening metadata (db_id, training_id, eco, name)
  - *Why*: Look up opening details when displaying recommendations
- **`holdout_players.csv`** (24 KB) - List of 1,000 holdout player IDs for testing
  - *Why*: Select test players that were never seen during training

### **⚠️ Missing Artifact (Need to Create)**
- **`opening_stats.json`** or **`.csv`** - Opening-specific statistics from training data
  - Should contain: `opening_id`, `mean_score`, `total_games`, `num_players`
  - *Why*: Required for hierarchical Bayesian shrinkage during inference
  - *Status*: **Not currently saved** - we'll need to generate this from notebook 28's training data

---

**Total artifact size**: <1MB currently. I'm not sure if opening_stats.json will be large and unwieldy (assuming we even need it) but I don't think so.

## Step 1: Load Test Player Data

We'll:
1. Load a player from the model artifacts (using training set for now - see note below)
2. Select a deterministic player (using a fixed index)
3. Extract their complete opening statistics from the database
4. Retrieve their player metadata
5. Display everything for verification

**Note**: Using a TRAINING player (not holdout) for initial testing because the holdout players list appears to include players without Black games. In production, we'll use the Lichess API to fetch any player's data, so this distinction won't matter. For now, we just need to validate the inference pipeline works.

**Deterministic Selection**: Using the same player index for consistency across runs.

In [1]:
# Standard library imports
import sys
from pathlib import Path
import json

# Third-party imports
import pandas as pd
import numpy as np

# Add project root to path for module imports
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

from utils.database.db_utils import get_db_connection

print("✓ All imports successful")

✓ All imports successful


In [ ]:
# Configuration
DB_PATH = PROJECT_ROOT / "data" / "processed" / "chess_games.db"
MODEL_ARTIFACTS_DIR = PROJECT_ROOT / "data" / "models" / "20251212_152017_black"

# Deterministic player selection
# Note: We'll find a player with actual Black games data
HOLDOUT_PLAYER_INDEX = 0  # Starting point, will find first player with Black games

# Verify paths exist
print(" Verifying paths...")
print(f"   Database: {DB_PATH.exists()} - {DB_PATH}")
print(f"   Model artifacts: {MODEL_ARTIFACTS_DIR.exists()} - {MODEL_ARTIFACTS_DIR}")

if not DB_PATH.exists():
    raise FileNotFoundError(f"Database not found at {DB_PATH}")
if not MODEL_ARTIFACTS_DIR.exists():
    raise FileNotFoundError(f"Model artifacts not found at {MODEL_ARTIFACTS_DIR}")

print("✓ All paths verified")

📂 Verifying paths...
   Database: True - /Users/a/Documents/personalprojects/chess-opening-recommender/data/processed/chess_games.db
   Model artifacts: True - /Users/a/Documents/personalprojects/chess-opening-recommender/data/models/20251212_152017_black
✓ All paths verified


In [ ]:
# Step 1.1: Load holdout players list
print("Loading holdout players...")

holdout_players_path = MODEL_ARTIFACTS_DIR / "holdout_players.csv"
holdout_players_df = pd.read_csv(holdout_players_path)

print(f"   Total holdout players: {len(holdout_players_df):,}")
print(f"   Columns: {list(holdout_players_df.columns)}")
print(f"\n   First 5 holdout players:")
print(holdout_players_df.head())

# Select our test player deterministically (always use the first player)
test_player_db_id = int(holdout_players_df.iloc[HOLDOUT_PLAYER_INDEX]['db_id'])
test_player_name = holdout_players_df.iloc[HOLDOUT_PLAYER_INDEX]['name']
test_player_rating = int(holdout_players_df.iloc[HOLDOUT_PLAYER_INDEX]['rating'])

print(f"\n✓ Selected holdout player #{HOLDOUT_PLAYER_INDEX}:")
print(f"   Database ID: {test_player_db_id}")
print(f"   Name: {test_player_name}")
print(f"   Rating: {test_player_rating}")

📋 Loading holdout players...
   Total holdout players: 1,000
   Columns: ['db_id', 'name', 'title', 'rating']

   First 5 holdout players:
   db_id        name  title  rating
0     53    AAashraf    NaN    1723
1    129       AK988    NaN    1489
2    187  ALexxxey89    NaN    1635
3    425   Abgrenzer    NaN    1787
4    498      Ac1000    NaN    2541

✓ Selected holdout player #0:
   Database ID: 53
   Name: AAashraf
   Rating: 1723


In [4]:
# Step 1.2: Verify player is NOT in training set
print("Verifying player is not in training set...")

player_mappings_path = MODEL_ARTIFACTS_DIR / "player_mappings.csv"
player_mappings_df = pd.read_csv(player_mappings_path)

print(f"   Total training players: {len(player_mappings_df):,}")
print(f"   Columns: {list(player_mappings_df.columns)}")

# Check if our test player is in the training set
is_in_training = test_player_db_id in player_mappings_df['db_id'].values

if is_in_training:
    raise ValueError(
        f"❌ ERROR: Player {test_player_name} (ID: {test_player_db_id}) "
        f"is in the training set! They should be a holdout player."
    )

print(f"✓ Verified: Player {test_player_name} (ID: {test_player_db_id}) is NOT in training set")
print(f"   This is a true holdout player for testing inference")

Verifying player is not in training set...
   Total training players: 48,551
   Columns: ['db_id', 'name', 'title', 'rating', 'training_id']
✓ Verified: Player AAashraf (ID: 53) is NOT in training set
   This is a true holdout player for testing inference


In [ ]:
# Step 1.3: Extract player's opening statistics from database
print(f"Extracting opening statistics for {test_player_name}...")

conn = get_db_connection(DB_PATH)

# Query to get all opening stats for this player (Black side, matching training data)
# Note: The schema uses 'color' column with values 'w' or 'b' (not 'white'/'black')
query = """
SELECT 
    pos.player_id,
    pos.opening_id,
    o.eco,
    o.name as opening_name,
    pos.num_wins + pos.num_draws + pos.num_losses as num_games,
    pos.num_wins as wins,
    pos.num_draws as draws,
    pos.num_losses as losses
FROM player_opening_stats pos
JOIN opening o ON pos.opening_id = o.id
WHERE pos.player_id = ?
  AND pos.color = 'b'
  AND (pos.num_wins + pos.num_draws + pos.num_losses) > 0
ORDER BY (pos.num_wins + pos.num_draws + pos.num_losses) DESC
"""

# Convert numpy.int64 to regular Python int for DuckDB compatibility
player_stats_df = conn.execute(query, [int(test_player_db_id)]).df()
conn.close()

print(f"✓ Database query complete")
print(f"   Total openings played: {len(player_stats_df)}")
print(f"   Total games: {player_stats_df['num_games'].sum():,}")
print(f"\n   Opening statistics columns: {list(player_stats_df.columns)}")
print(f"   Data types:\n{player_stats_df.dtypes}")

📊 Extracting opening statistics for AAashraf...
✓ Database query complete
   Total openings played: 194
   Total games: 3,502

   Opening statistics columns: ['player_id', 'opening_id', 'eco', 'opening_name', 'num_games', 'wins', 'draws', 'losses']
   Data types:
player_id        int32
opening_id       int32
eco             object
opening_name    object
num_games        int32
wins             int32
draws            int32
losses           int32
dtype: object
✓ Database query complete
   Total openings played: 194
   Total games: 3,502

   Opening statistics columns: ['player_id', 'opening_id', 'eco', 'opening_name', 'num_games', 'wins', 'draws', 'losses']
   Data types:
player_id        int32
opening_id       int32
eco             object
opening_name    object
num_games        int32
wins             int32
draws            int32
losses           int32
dtype: object


In [ ]:
# Step 1.4: Display complete player data for verification
print("="*80)
print(f"COMPLETE DATA FOR TEST PLAYER: {test_player_name}")
print("="*80)

print("\nPLAYER METADATA:")
print(f"   Database ID: {test_player_db_id}")
print(f"   Name: {test_player_name}")
print(f"   Rating: {test_player_rating}")
print(f"   Status: HOLDOUT PLAYER (not in training set)")

print("\nOPENING STATISTICS SUMMARY:")
print(f"   Total openings played: {len(player_stats_df)}")
print(f"   Total games: {player_stats_df['num_games'].sum():,}")
print(f"   Total wins: {player_stats_df['wins'].sum()}")
print(f"   Total draws: {player_stats_df['draws'].sum()}")
print(f"   Total losses: {player_stats_df['losses'].sum()}")
print(f"   Overall win rate: {player_stats_df['wins'].sum() / player_stats_df['num_games'].sum():.1%}")

print("\nTOP 10 MOST-PLAYED OPENINGS:")
print(player_stats_df[['opening_name', 'eco', 'num_games', 'wins', 'draws', 'losses']].head(10).to_string(index=False))

print("\nBOTTOM 10 LEAST-PLAYED OPENINGS:")
print(player_stats_df[['opening_name', 'eco', 'num_games', 'wins', 'draws', 'losses']].tail(10).to_string(index=False))

print("\nVERIFICATION COMPLETE")
print(f"   Player data extracted successfully for {test_player_name}")
print(f"   Ready to proceed to data transformation (Step 2)")
print("="*80)

COMPLETE DATA FOR TEST PLAYER: AAashraf

📋 PLAYER METADATA:
   Database ID: 53
   Name: AAashraf
   Rating: 1723
   Status: HOLDOUT PLAYER (not in training set)

📊 OPENING STATISTICS SUMMARY:
   Total openings played: 194
   Total games: 3,502
   Total wins: 1531
   Total draws: 292
   Total losses: 1679
   Overall win rate: 43.7%

📈 TOP 10 MOST-PLAYED OPENINGS:
                                                           opening_name eco  num_games  wins  draws  losses
                                                       Philidor Defense C41       1450   642    119     689
                           Queen's Pawn Game: Accelerated London System D00        206    99     20      87
                                        Bishop's Opening: Boi Variation C23        136    64     11      61
                                                            Vienna Game C25        127    61     13      53
                                          Queen's Pawn Game: Anti-Torre D02         98    46   